# Vanadium processing

The main workflow in [POWGEN_data_reduction](./POWGEN_data_reduction.rst) misses some reduction steps for vanadium.
In particular, we need to remove coherent scattering peaks from vanadium data.
This is not part of the regular workflow as it relies on fitting.
And since fitting can easily break in a way that is hard to detect automatically, a human should inspect the results.
Additionally, vanadium measurements can be processed separately from sample measurements and saved to files.
The processed vanadium data can then be used in the main workflow directly.

This notebook outlines how to process a vanadium run.
First, we convert the data to d-spacing using the same workflow as in [POWGEN_data_reduction](./POWGEN_data_reduction.rst).

In [ ]:
import scipp as sc
import scippneutron as scn
import scippneutron.peaks
import sciline

from ess import powder
from ess.powder.external import powgen
from ess.powder.types import *

Use the same parameters as in the main workflow except with more d-spacing bins.
We need the high d-spacing resolution later when removing coherent scattering peaks.

In [ ]:
params = {
    NeXusDetectorName: "powgen_detector",
    # Input data
    Filename[VanadiumRun]: powgen.data.powgen_tutorial_vanadium_file(),
    # Edges for binning in d-spacing
    DspacingBins: sc.linspace("dspacing", 0.0, 2.3434, 5001, unit="angstrom"),
    # Mask in time-of-flight to crop to valid range
    TofMask: lambda x: (x < sc.scalar(0.0, unit="us"))
    | (x > sc.scalar(16666.67, unit="us")),
    TwoThetaMask: None,
    WavelengthMask: None,
}

In [ ]:
providers = [*powder.providers, *powgen.providers]
pipeline = sciline.Pipeline(providers, params=params)
pipeline = powder.with_pixel_mask_filenames(pipeline, [])

In [ ]:
pipeline.visualize(FocussedDataDspacing[VanadiumRun], graph_attr={"rankdir": "LR"})

Compute a single vanadium spectrum in d-spacing:

In [ ]:
peaked_data = pipeline.compute(FocussedDataDspacing[VanadiumRun])

In [ ]:
peaked_data

In [ ]:
peaked_data.hist().plot()

## Removing coherent scattering peaks

As the variable name `peaked_data` implies, the produced spectrum contains peaks from coherent scattering.
Even though the peaks are small for vanadium, we need to remove them to extract pure incoherent scattering.
We can approximate the coherent scattering contribution by fitting functions to the peaks and subtracting those fitted functions.
[scippneutron.peaks](https://scipp.github.io/scippneutron/generated/modules/scippneutron.peaks.html) contains general functionality for fitting and removing peaks.
Here, we use it through [ess.powder.external.powgen.peaks](../../generated/modules/ess.powder.external.powgen.peaks.rst) which provides useful defaults for vanadium peaks at POWGEN.
For example, it selects appropriate models for peaks (gaussian) and backgrounds (linear and quadratic).

First, define estimates for the peaks based on the known crystal structure of vanadium:

In [ ]:
peak_estimates = powgen.peaks.theoretical_vanadium_dspacing(
    hkl_range=7, min_d=sc.scalar(0.41, unit='angstrom')
)

We need to histogram the data to perform fits:

In [ ]:
peak_histogram = peaked_data.hist()

The fits require a bin-center coordinate, so convert from bin-edges:

In [ ]:
to_fit = peak_histogram.copy(deep=False)
to_fit.coords['dspacing'] = sc.midpoints(to_fit.coords['dspacing'])

Perform the fits:

In [ ]:
fit_results = powgen.peaks.fit_vanadium_peaks(to_fit, peak_estimates=peak_estimates)

Remove the fitted peaks to obtain the incoherent scattering.
Also restore the bin-edge coordinate that we had to replace temporarily for the fits.

Importantly, we remove variances from the data.
If we kept the variances, subtracting the fitted models would introduce correlations between the data points.
This corresponds to [UncertaintyBroadcastMode.drop](../../generated/modules/ess.powder.types.UncertaintyBroadcastMode.rst) in the main workflow.
See also the [guide in ESSreduce](https://scipp.github.io/essreduce/user-guide/reduction-workflow-guidelines.html#s-8-propagation-of-uncertainties-in-broadcast-operations-should-support-drop-and-upper-bound-strategies-upper-bound-shall-be-the-default)

In [ ]:
incoherent = scn.peaks.remove_peaks(sc.values(to_fit), fit_results)
incoherent.coords['dspacing'] = peak_histogram.coords['dspacing']
incoherent.plot()

We can further inspect the results.
Below, there is a function that plots
- the data with coherent and incoherent scattering (blue),
- the resulting incoherent curve (green),
- the fitted models (orange),
- the fit windows (gray and red boxes),
- and the initial estimates (dashed vertical lines).

Some fits failed as indicated by red boxes and short descriptions of why the fits failed.
Some peaks are absent from the data used here, even though they are expected based on the crystal structure.
So those fits are expected to fail.
All other fits appear to have succeeded.

See [scippneutron.peaks.fit_peaks](https://scipp.github.io/scippneutron/generated/modules/scippneutron.peaks.fit_peaks.html) for options to customize the fit procedure if it does not work as desired.

<div class="alert alert-info">

**Note**

It is highly recommended to inspect the plot in detail to check whether all fits have succeeded or failed as expected!
Fitting is not always reliable and may fail for many reasons.
You can make plots interactive by using `%matplotlib widget`.

</div>

In [ ]:
def peak_removal_diagnostic(
    data: sc.DataArray,
    removed: sc.DataArray,
    fit_results: list[scn.peaks.FitResult],
    peak_estimates: sc.Variable,
    *,
    xlim: tuple[sc.Variable, sc.Variable] | None=None,
):
    # The actual data
    fig = data.plot(c="C0")
    ax = fig.ax
    removed.plot(ax=ax, c='C2')

    # Initial estimates
    for estimate, result in zip(peak_estimates, fit_results, strict=True):
        ax.axvline(
            x=estimate.value,
            color="black" if result.success else "red",
            alpha=0.5,
            lw=1,
            ls=":",
        )

    # Fit windows
    for result in fit_results:
        left = result.window[0]
        right = result.window[1]
        sl = data[data.dim, left:right]
        lo = sl.min().value * 0.95
        hi = sl.max().value * 1.05
        ax.fill_betweenx(
            (lo, hi),
            left.value,
            right.value,
            facecolor="black" if result.success else "red",
            alpha=0.2,
        )
        if not result.success:
            ax.text(left.value, hi, result.message.split(":", 1)[0])

    # Overlay with fit models evaluated at optimized parameters
    for result in fit_results:
        if all(not sc.isnan(param).value for param in result.popt.values()):
            best_fit = data[data.dim, result.window[0] : result.window[1]].copy(deep=False)
            best_fit.coords[best_fit.dim] = sc.midpoints(best_fit.coords[best_fit.dim])
            best_fit.data = result.eval_model(best_fit.coords[best_fit.dim])
            best_fit.plot(ax=ax, c="C1", ls="-", marker="none")

    if xlim is not None:
        x_unit = data.coords[data.dim].unit
        ax.set_xlim((xlim[0].to(unit=x_unit).value,
                     xlim[1].to(unit=x_unit).value))
        in_lim = data[data.dim, xlim[0]:xlim[1]]
        ax.set_ylim((in_lim.min().value*0.97, in_lim.max().value*1.06))

    return fig

In [ ]:
peak_removal_diagnostic(
    peak_histogram,
    incoherent,
    fit_results,
    peak_estimates,
)

In [ ]:
peak_removal_diagnostic(
    peak_histogram,
    incoherent,
    fit_results,
    peak_estimates,
    xlim=(0.37 * sc.Unit("Å"), 0.56 * sc.Unit("Å")),
)

The resulting data array `incoherent` can be saved and used in the main workflow [POWGEN_data_reduction](./POWGEN_data_reduction.rst) to replace `FocussedDataDspacing[VanadiumRun]`.